# Natural Language Processing with Attention Models

Notas sobre o curso Natural Language Processing with Attention Models da DeeplearninigAI. O notebook é composto majoritariamente de material original, salvo as figuras, que foram criadas pela **Deep Learning AI** e disponibilizadas em seu curso.

# Week 1 - Neural Machine Translation

## Seq2seq

Os modelos Seq2Seq (Sequence-to-Sequence) são amplamente utilizados em tarefas de tradução automática (machine translation) e outras tarefas de NLP que envolvem a conversão de uma sequência de entrada em uma sequência de saída. Eles foram inicialmente propostos para tarefas como tradução automática, onde uma sequência de palavras em uma língua (entrada) é convertida em uma sequência de palavras em outra língua (saída). A arquitetura básica é composta por dois componentes principais:

- **Encoder**: Transforma a sequência de entrada em um vetor de contexto.
- **Decoder**: Utiliza o vetor de contexto para gerar a sequência de saída.

<img src="./imgs/seq2seq.png">

### Arquitetura Seq2Seq

#### Encoder

O encoder é tipicamente uma rede recorrente (RNN, LSTM, GRU) que lê a sequência de entrada token por token e gera um vetor de contexto que representa a informação da sequência inteira.

- **Inputs**: Sequência de entrada $X = (x_1, x_2, ..., x_n)$.
- **Outputs**: Vetor de contexto $C$ e estado oculto $h_t$ em cada passo de tempo.

<img src="./imgs/seq2seq_encoder.png">

#### Decoder

O decoder é outra rede recorrente que gera a sequência de saída um token por vez, **usando o vetor de contexto do encoder** e o **estado oculto anterior do decoder**. A sequencia de entrada no decoder inicia com token $\text{<SOS>}$ (Start of Sequence)

- **Inputs**: O vetor de contexto do encoder, o estado oculto anterior e o token de entrada anterior.
- **Outputs**: Token de saída e novo estado oculto.

<img src="./imgs/seq2seq_decoder.png">

#### Attention

O mecanismo de atenção foi introduzido para melhorar a performance dos modelos Seq2Seq, permitindo que o modelo foque em diferentes partes da entrada enquanto gera cada token da saída.

- **Atenção Básica**: Calcula um conjunto de pesos de atenção que indicam a importância de cada token da sequência de entrada para a geração do próximo token da saída.
- **Context Vector**: Um vetor de contexto ponderado é calculado como uma combinação linear dos estados ocultos do encoder, ponderado pelos pesos de atenção.

Matematicamente, a atenção pode ser descrita da seguinte maneira:
$$ \alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k=1}^{T_x} \exp(e_{ik})} $$
onde $e_{ij} = a(s_{i-1}, h_j)$ é a função de alinhamento que pode ser um simples perceptron.

<img src="./imgs/seq2seq_attention.png">

### Aplicação em Machine Translation

Na tradução automática, o processo funciona da seguinte maneira:

1. **Encoding**: O encoder lê a sequência de entrada (frase na língua de origem) e gera um vetor de contexto.
2. **Attention**: Durante a decodificação, o mecanismo de atenção calcula os pesos de atenção que focam nas partes relevantes da sequência de entrada.
3. **Decoding**: O decoder gera a sequência de saída (frase na língua de destino), token por token, utilizando o vetor de contexto ponderado pela atenção.

### Exemplo Simplificado

1. **Entrada**: "I love NLP"
2. **Encoding**: O encoder gera vetores de estado oculto para cada token e um vetor de contexto geral.
3. **Attention**: Para cada token gerado na saída, calcula-se a importância de cada token da entrada.
4. **Decoding**: Gera a saída "Yo amo PLN" utilizando os vetores de contexto e atenção.

### Benefícios do Mecanismo de Atenção

- **Melhora a Tradução**: Permite que o modelo foque nas partes mais relevantes da entrada.
- **Mitiga Problemas de Longas Sequências**: Ajuda a manter a informação relevante mesmo em sequências longas.

Os modelos Seq2Seq com atenção são fundamentais para muitas tarefas de NLP, especialmente a tradução automática. A arquitetura básica de encoder-decoder, aprimorada com o mecanismo de atenção, permite que esses modelos lidem eficazmente com a complexidade das sequências de linguagem natural.

## Queries, Keys, Values, and Attention

A camada de atenção calcula um **vetor de atenção que pondera a importância de cada token da sequência de entrada para a geração de cada token da sequência de saída**. 

#### Inputs da Camada de Atenção

A camada de atenção recebe três inputs principais:
- **Queries (Q)**: Vetores de consulta que representam os tokens da **sequência de saída** atual.
- **Keys (K)**: Vetores de chave que representam os tokens da **sequência de entrada**.
- **Values (V)**: Vetores de valor que também representam os tokens da **sequência de entrada**.

<img src="./imgs/qkv.png">

Em um modelo Seq2Seq com atenção, **os estados ocultos do decoder são usados como Queries**, enquanto **os estados ocultos do encoder são usados como Keys e Values**.

#### Cálculo das Similaridades (Scores)

Primeiro, calculamos uma pontuação (score) ou alinhamento que mede a **similaridade entre cada par de query e key**. Uma função de similaridade comum é o produto escalar (dot product):

$$ \text{score}(Q, K) = QK^T $$

Outra opção é o produto escalar escalado (scaled dot product), que é comum em Transformers:

$$ \text{score}(Q, K) = \frac{QK^T}{\sqrt{d_k}} $$

onde $ d_k $ é a dimensão dos vetores de chave.

#### Normalização dos scores

As pontuações escaladas são então normalizadas usando a função softmax para obter os pesos de atenção:

$$ \alpha = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) $$

Isso transforma as pontuações escaladas em um conjunto de pesos que somam 1, facilitando a interpretação como probabilidades.

#### Cálculo dos Pesos de Atenção

As pontuações são então normalizadas usando uma função softmax para obter os pesos de atenção, que representam a importância relativa de cada token de entrada para cada token de saída:

$$ \alpha_{ij} = \frac{\exp(\text{score}(q_i, k_j))}{\sum_{k=1}^{n} \exp(\text{score}(q_i, k_k))} $$

onde $ \alpha_{ij} $ é o peso de atenção para o $ i $-ésimo token de saída e o $ j $-ésimo token de entrada.

#### Cálculo do Vetor de Contexto

Finalmente, os pesos de atenção são usados para calcular uma combinação ponderada dos vetores de valor:

$$ \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V $$

O resultado é uma matriz que representa a combinação ponderada dos valores, ajustada de acordo com a relevância calculada pelas pontuações de atenção.

Os pesos de atenção são então usados para calcular uma **combinação ponderada dos vetores de valor**, resultando em um vetor de contexto para cada token de saída:

$$ \text{context}_i = \sum_{j=1}^{n} \alpha_{ij} v_j $$

#### Atenção Multi-cabeça (Multi-head Attention)

Em arquiteturas como Transformers, é comum usar várias cabeças de atenção para capturar diferentes tipos de relações entre tokens. Cada cabeça de atenção realiza o processo descrito acima de forma independente, e os resultados são concatenados e transformados:

$$ \text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \text{head}_2, ..., \text{head}_h)W^O $$

onde cada cabeça de atenção $ \text{head}_i $ é calculada como:

$$ \text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V) $$

### Tipos de Atenção

Existem diferentes variações do mecanismo de atenção, incluindo:

1. **Self-attention**: Quando Queries, Keys e Values vêm da mesma sequência. Utilizado principalmente em Transformers.
2. **Cross-attention**: Quando Queries vêm de uma sequência (ex. saída) e Keys/Values vêm de outra (ex. entrada). Utilizado em modelos Seq2Seq com atenção.

### Implementação Simplificada

Aqui está um exemplo de implementação simplificada de uma camada de atenção em pseudo-código:

```python
def attention(Q, K, V):
    # Cálculo das pontuações
    scores = dot_product(Q, K.T) / sqrt(d_k)
    
    # Normalização das pontuações
    attention_weights = softmax(scores, axis=-1)
    
    # Cálculo do vetor de contexto
    context = dot_product(attention_weights, V)
    
    return context, attention_weights
```

### Benefícios da Camada de Atenção

- **Foco Dinâmico**: Permite que o modelo se concentre nas partes mais relevantes da entrada.
- **Eficiência Computacional**: Self-attention, especialmente em Transformers, é altamente paralelizável.
- **Flexibilidade**: Pode ser adaptada para várias tarefas de NLP, como tradução, resumo e resposta a perguntas.

### Conclusão

A camada de atenção é um componente poderoso que melhora a capacidade dos modelos de NLP em capturar dependências complexas dentro das sequências de entrada. Ao permitir que o modelo foque dinamicamente nas partes mais relevantes da entrada, a atenção desempenha um papel crucial em muitos dos avanços recentes em NLP.

## Material complementar
- [Visualizando a atenção, o coração de um transformador](https://www.youtube.com/watch?v=eMlx5fFNoYc)
- [Atenção para Redes Neurais, Claramente Explicadas!!!](https://www.youtube.com/watch?v=PSs6nxngL6k)
- [The math behind Attention: Keys, Queries, and Values matrices](https://www.youtube.com/watch?v=UPtG_38Oq8o&t=669s)
- [The Transformer neural network architecture EXPLAINED. “Attention is all you need”](https://www.youtube.com/watch?v=FWFA4DGuzSc)

## Setup for Machine Translation

## Teacher Forcing

## NMT Model with Attention

## BLEU Score

## ROUGE-N Score

## Sampling and Decoding

## Beam Search

## Minimum Bayes Risk

## Ungraded Lab: Stack Semantics

## Neural Machine Translation

## NMT with Attention

# Week 2

# Week 3

# Week 4

# Referência
- Natural Language Processing with Attention Models, disponível em https://www.coursera.org/learn/probabilistic-models-in-nlp

# Licença
- CC BY-SA 2.0 LEGAL CODE. Attribution-ShareAlike 2.0 Generic
- Para detalhes sobre a licença, verifique https://creativecommons.org/licenses/by-sa/2.0/legalcode